In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3`


import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window



//Reducir numero logs
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)


//For adding extra dependenies
import $ivy.`org.typelevel::cats-core:1.6.0`

//Plotly
import $ivy.`org.plotly-scala::plotly-almond:0.8.3`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._


//Iniciamos session de Spark
val spark = {
  NotebookSparkSession.builder()
     .master("local[*]")
    //.config("spark.executor.instances", "4") // Número de ejecutores
    //.config("spark.executor.memory", "4g") // Memoria por ejecutor
    //.config("spark.executor.cores", "2") // Núcleos por ejecutor
    //.config("spark.driver.memory", "4g") // Memoria del driver
    //.config("spark.sql.shuffle.partitions", "8") // Número de particiones para operaciones de shuffle
    .getOrCreate()
}
import spark.implicits._

Loading spark-stubs

Creating SparkSession

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import org.apache.log4j.{Level, Logger}
import $ivy.$
import $ivy.$
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._


//Iniciamos session de Spark

spark: SparkSession = org.apache.spark.sql.SparkSession@50226ec8
import spark.implicits._

In [2]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.Row
val itemsSchema = StructType(Array(
    StructField("id", IntegerType, false),
    StructField("name", StringType, false),
    StructField("price", FloatType, false)
))

val ordersSchema = StructType(Array(
    StructField("id", IntegerType, false),
    StructField("itemid", IntegerType, false),
    StructField("count", IntegerType, false)
))

// 📌 Crear listas de datos
val itemsData = Seq(
    Row(0, "Tomato", 2.0f),
    Row(1, "Watermelon", 5.5f),
    Row(2, "Pineapple", 7.0f)
)

val ordersData = Seq(
    Row(100, 0, 1),
    Row(100, 1, 1),
    Row(101, 2, 3),
    Row(102, 2, 8)
)

// 📌 Crear DataFrames
val itemsDF = spark.createDataFrame(spark.sparkContext.parallelize(itemsData), itemsSchema)
val ordersDF = spark.createDataFrame(spark.sparkContext.parallelize(ordersData), ordersSchema)

// 📌 Mostrar los DataFrames
itemsDF.show()
ordersDF.show()

+---+----------+-----+
| id|      name|price|
+---+----------+-----+
|  0|    Tomato|  2.0|
|  1|Watermelon|  5.5|
|  2| Pineapple|  7.0|
+---+----------+-----+



+---+------+-----+
| id|itemid|count|
+---+------+-----+
|100|     0|    1|
|100|     1|    1|
|101|     2|    3|
|102|     2|    8|
+---+------+-----+



import org.apache.spark.sql.types._
import org.apache.spark.sql.Row
itemsSchema: StructType = StructType(
  StructField("id", IntegerType, false, {}),
  StructField("name", StringType, false, {}),
  StructField("price", FloatType, false, {})
)
ordersSchema: StructType = StructType(
  StructField("id", IntegerType, false, {}),
  StructField("itemid", IntegerType, false, {}),
  StructField("count", IntegerType, false, {})
)
itemsData: Seq[Row] = List(
  [0,Tomato,2.0],
  [1,Watermelon,5.5],
  [2,Pineapple,7.0]
)
ordersData: Seq[Row] = List([100,0,1], [100,1,1], [101,2,3], [102,2,8])
itemsDF: DataFrame = [id: int, name: string ... 1 more field]
ordersDF: DataFrame = [id: int, itemid: int ... 1 more field]

In [4]:
import org.apache.spark.sql.functions._

val y= itemsDF
    .join(ordersDF, itemsDF("id") === ordersDF("itemid"), "inner")
    .where(itemsDF("id") === 2)
    .groupBy("name","price")
    .agg(sum("count").alias("c")) 


y.show()


+---------+-----+---+
|     name|price|  c|
+---------+-----+---+
|Pineapple|  7.0| 11|
+---------+-----+---+



import org.apache.spark.sql.functions._
y: DataFrame = [name: string, price: float ... 1 more field]

In [5]:
y.explain

== Physical Plan ==
*(6) HashAggregate(keys=[name#4, price#5], functions=[sum(cast(count#14 as bigint))])
+- Exchange hashpartitioning(name#4, price#5, 200)
   +- *(5) HashAggregate(keys=[name#4, price#5], functions=[partial_sum(cast(count#14 as bigint))])
      +- *(5) Project [name#4, price#5, count#14]
         +- *(5) SortMergeJoin [id#3], [itemid#13], Inner
            :- *(2) Sort [id#3 ASC NULLS FIRST], false, 0
            :  +- Exchange hashpartitioning(id#3, 200)
            :     +- *(1) Filter (id#3 = 2)
            :        +- Scan ExistingRDD[id#3,name#4,price#5]
            +- *(4) Sort [itemid#13 ASC NULLS FIRST], false, 0
               +- Exchange hashpartitioning(itemid#13, 200)
                  +- *(3) Project [itemid#13, count#14]
                     +- *(3) Filter (itemid#13 = 2)
                        +- Scan ExistingRDD[id#12,itemid#13,count#14]
